In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
from keras.datasets import cifar10

In [3]:
(train_ds, validation_ds, test_ds), info = tfds.load('cifar10',
                                    split=["train[:50%]", "train[50%:70%]", "train[70%:]"],
                                    as_supervised = True,
                                    with_info=True,
                                    shuffle_files= True)

In [4]:
size = (32, 32)
batch_size = 64

def normalize_resize(image, label):
    image = tf.cast(image, tf.float32)
    image = tf.divide(image, 255)
    image = tf.image.resize(image, size)
    return image, label

def augment(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_saturation(image, 0.7, 1.3)
    image = tf.image.random_contrast(image, 0.8, 1.2)
    image = tf.image.random_brightness(image, 0.1)
    image = tf.image.random_crop(image, (32, 32, 3))
    return image, label 

train_ds = train_ds.map(normalize_resize).cache().map(augment).shuffle(100000).batch(batch_size).prefetch(buffer_size=1)
validation_ds = validation_ds.map(normalize_resize).cache().batch(batch_size).prefetch(buffer_size=1)
test_ds = test_ds.map(normalize_resize).cache().batch(batch_size).prefetch(buffer_size=1)

In [5]:
%cd  /home/reynier/py_projects/Deployments/functions

/home/reynier/py_projects/Deployments/functions


In [6]:
from Nets import GetNeuralNetworkModel
from Stats import WeightQuantization, ActivationStats, CheckAccuracyAndLoss, QuantizationEffect, GetReadAndWrites
from funciones import buffer_vectores
from Simulation import save_obj, load_obj

In [7]:
model = GetNeuralNetworkModel('ResNet',(32,32,3),10, quantization = False, aging_active=False)

In [8]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 rescaling (Rescaling)          (None, 32, 32, 3)    0           ['input_1[0][0]']                
                                                                                                  
 lambda (Lambda)                (None, 32, 32, 3)    0           ['rescaling[0][0]']              
                                                                                                  
 lambda_1 (Lambda)              (None, 32, 32, 3)    0           ['lambda[0][0]']                 
                                                                                              

 batch_normalization_7 (BatchNo  (None, 7, 7, 256)   1024        ['Conv3_2[0][0]']                
 rmalization)                                                                                     
                                                                                                  
 activation_6 (Activation)      (None, 7, 7, 256)    0           ['batch_normalization_7[0][0]']  
                                                                                                  
 lambda_12 (Lambda)             (None, 7, 7, 256)    0           ['activation_6[0][0]']           
                                                                                                  
 add_1 (Add)                    (None, 7, 7, 256)    0           ['lambda_12[0][0]',              
                                                                  'activation_3[0][0]']           
                                                                                                  
 activatio

 batch_normalization_15 (BatchN  (None, 4, 4, 128)   512         ['Conv1_5[0][0]']                
 ormalization)                                                                                    
                                                                                                  
 activation_15 (Activation)     (None, 4, 4, 128)    0           ['batch_normalization_15[0][0]'] 
                                                                                                  
 lambda_24 (Lambda)             (None, 4, 4, 128)    0           ['activation_15[0][0]']          
                                                                                                  
 Conv2_5 (Conv2D)               (None, 4, 4, 128)    147584      ['lambda_24[0][0]']              
                                                                                                  
 batch_normalization_16 (BatchN  (None, 4, 4, 128)   512         ['Conv2_5[0][0]']                
 ormalizat

 ormalization)                                                                                    
                                                                                                  
 activation_25 (Activation)     (None, 4, 4, 512)    0           ['batch_normalization_23[0][0]'] 
                                                                                                  
 lambda_34 (Lambda)             (None, 4, 4, 512)    0           ['activation_25[0][0]']          
                                                                                                  
 add_6 (Add)                    (None, 4, 4, 512)    0           ['lambda_34[0][0]',              
                                                                  'activation_22[0][0]']          
                                                                                                  
 activation_26 (Activation)     (None, 4, 4, 512)    0           ['add_6[0][0]']                  
          

 ormalization)                                                                                    
                                                                                                  
 activation_34 (Activation)     (None, 2, 2, 256)    0           ['batch_normalization_31[0][0]'] 
                                                                                                  
 lambda_46 (Lambda)             (None, 2, 2, 256)    0           ['activation_34[0][0]']          
                                                                                                  
 Conv2_10 (Conv2D)              (None, 2, 2, 256)    590080      ['lambda_46[0][0]']              
                                                                                                  
 batch_normalization_32 (BatchN  (None, 2, 2, 256)   1024        ['Conv2_10[0][0]']               
 ormalization)                                                                                    
          

                                                                                                  
 activation_44 (Activation)     (None, 2, 2, 1024)   0           ['batch_normalization_39[0][0]'] 
                                                                                                  
 lambda_56 (Lambda)             (None, 2, 2, 1024)   0           ['activation_44[0][0]']          
                                                                                                  
 add_11 (Add)                   (None, 2, 2, 1024)   0           ['lambda_56[0][0]',              
                                                                  'activation_41[0][0]']          
                                                                                                  
 activation_45 (Activation)     (None, 2, 2, 1024)   0           ['add_11[0][0]']                 
                                                                                                  
 Conv1_13 

                                                                                                  
 activation_53 (Activation)     (None, 1, 1, 512)    0           ['batch_normalization_47[0][0]'] 
                                                                                                  
 lambda_68 (Lambda)             (None, 1, 1, 512)    0           ['activation_53[0][0]']          
                                                                                                  
 Conv2_15 (Conv2D)              (None, 1, 1, 512)    2359808     ['lambda_68[0][0]']              
                                                                                                  
 batch_normalization_48 (BatchN  (None, 1, 1, 512)   2048        ['Conv2_15[0][0]']               
 ormalization)                                                                                    
                                                                                                  
 activatio

In [9]:
callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
    )

In [10]:
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy

In [11]:
model.compile(
    optimizer=Adam(0.00001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'],
)

In [12]:
#%cd /home/reynier/py_projects/Deployments/CIFAR/Weights/ResNet/

In [13]:
#model.load_weights('weights.data')

In [ ]:
epochs = 500
history = model.fit(train_ds, epochs=epochs, validation_data=validation_ds, callbacks=[callback])

Epoch 1/500
391/391 [==============================] - 77s 185ms/step - loss: 18.5150 - accuracy: 0.1199 - val_loss: 16.7783 - val_accuracy: 0.1219
Epoch 2/500
391/391 [==============================] - 73s 184ms/step - loss: 15.2872 - accuracy: 0.1499 - val_loss: 13.8588 - val_accuracy: 0.1677
Epoch 3/500
391/391 [==============================] - 73s 184ms/step - loss: 12.7472 - accuracy: 0.1789 - val_loss: 11.7128 - val_accuracy: 0.2065
Epoch 4/500
391/391 [==============================] - 73s 184ms/step - loss: 10.9087 - accuracy: 0.2044 - val_loss: 10.1337 - val_accuracy: 0.2169
Epoch 5/500
391/391 [==============================] - 74s 185ms/step - loss: 9.5127 - accuracy: 0.2144 - val_loss: 8.9722 - val_accuracy: 0.2020
Epoch 6/500
391/391 [==============================] - 73s 184ms/step - loss: 8.4162 - accuracy: 0.2265 - val_loss: 7.9463 - val_accuracy: 0.2379
Epoch 7/500
391/391 [==============================] - 74s 186ms/step - loss: 7.5463 - accuracy: 0.2358 - val_loss: 

In [ ]:
score = model.evaluate(test_ds, verbose=1)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
score = model.evaluate(train_ds, verbose=1)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
#%cd /home/reynier/py_projects/Deployments/CIFAR/Weights/ResNet/V1

In [ ]:
%cd /home/yamilka/Yamilka/Trained_Weights_cifar/ResNet

In [ ]:
model.save_weights('weights.data')